In [1]:
!pip install konlpy  # 토큰화에 사용할 konlpy 라이브러리 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 32.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import os, random
from tqdm import tqdm # 진행도 시각화를 위한 라이브러리

seed=42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

# 데이터 불러오기

In [3]:
train_data = pd.read_csv("/kaggle/input/2023-ml-project1/nsmc_train.csv", index_col=0)
test_data = pd.read_csv("/kaggle/input/2023-ml-project1/nsmc_test.csv", index_col=0)
print(train_data.shape)
print(test_data.shape)

(149993, 2)
(49999, 1)


In [4]:
train_data.head()

,review,rating
id,,
9324809,배우들의 인생연기가 돋보였던... 최고의 드라마,1
9305425,아 혜리 보고싶다 ... 여군좀 ㅠ,0
5239110,"눈이 팅팅..... 정말 ,..... 대박이다......",1
9148159,캐슬린 터너의 보디는 볼만했다,0
6144938,진짜 최고였다.,1


In [5]:
x_train = train_data["review"]
y_train = np.array(train_data["rating"])

# 자연어 전처리
## \[Empty Module #1\] 데이터 전처리
### 데이터 전처리 수행
> 먼저, 리뷰를 분류하는데 도움이 되거나, 머신러닝 처리에 어려운 단어들을 제거해봅시다.
1. 아래 조건에 맞는 정규표현식을 작성하여 영어와 한글 문자를 제외한 특수문자나 이모지, 숫자 등을 제거해봅시다.
  - <mark>한글 문자(초성 제외), 영어 대문자, 영어 소문자, 띄어쓰기 이외의 문자를 제외</mark>하는 정규표현식 작성
2. 영어 단어의 경우 같은 단어들이 같은 토큰으로 분류될 수 있도록 <mark>대문자로 통일</mark>해줍니다.

In [6]:
##########################################################################################
# Empty Module #1
# 입력: 자연어 상태의 리뷰 텍스트
# 출력: 한글(초성 제외), 영어 대문자, 띄어쓰기로만 구성된 텍스트 
# 입력 예시: "안녕 Hello!!!:)ㅎㅎ반갑다."
# 출력 예시: "안녕 HELLO반갑다"
##########################################################################################
import re

# 여기에 정규표현식 코드 작성
pattern = r'[^가-힣a-zA-Z\s]'#"^" : 제외, "가-힣": 한글의 모든 코드(초성 제외), "a-zA-Z": 대문자, 소문자, "\s": 공백 문자

def apply_regex(pattern, text):  # 정규표현식을 이용한 필터링 적용
    text = re.sub(pattern, " ", text)  # 정규표현식 패턴에 맞는 값들을 텍스트에서 제거
    text = text.upper() # 영어들을 찾아 대문자로 치환하는 코드 작성

    return text

x_train_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_train.iteritems(), total=len(x_train), desc="pre-processing data")]

/tmp/ipykernel_20/549623988.py:19: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  x_train_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_train.iteritems(), total=len(x_train), desc="pre-processing data")]
pre-processing data: 100%|██████████| 149993/149993 [00:00<00:00, 305602.44it/s]


In [7]:
#잘 처리 되었는지 확인
x_train_preprocessed[60:70]

['사람은 태어나면 죽는다   영화를 찍은 배우들의 삶의 시간이 아까울정도다 ',
 '무슨 마약하셨길래 이런생각을 했어요 ',
 '영애씨 재밌어요계속기대됩니다 ',
 '김지영넘불상해서 보기싫어지는 드라마다 현지X년 연기정말 못한다 꼴보기싫타 생긴것도 다성형한것같아 미친XXX년같다 당정하차시켜라  작가는 더미친XXX다 좀 잘쓰라',
 '강시라는 엄청난 소재를 가지고 이렇게 밖에 못만들다니   어릴적 강시에 대한 기대감으로 봤는데 실망이 이만저만    한정적인 공간과 본연의 강시의 공포를 끌어내지도 못하고 터무니 없는 액션으로 쓰레기 영화 인증해버림',
 '마녀 이후로 티비엔 호청자가 됐긔 그래서 로필   나인 전부 다봤긔',
 '영화관에서볼만한건아니였지만 재밌었음 후반에 실망감이 조금',
 '재미없다 ',
 '정말멋진지구다큐맨터리 화질정말끝내주네   ',
 '이거슨 명작이다   ']

## \[Empty Module #2\] 단어 토큰화
### Open Korean Text(OKT)를 이용한 단어 토큰화(Tokenization)
- 한국어 자연어 처리 라이브러리인 konlpy의 OKT 래퍼를 통하여 문장을 단어로 토큰화해봅시다.
- 아래 도큐먼트를 참고하여 토큰화를 수행합니다.
  - <mark>이때, OKT 클래스의 특정 매개변수를 이용해 어근화를 진행해줍니다. (도큐먼트 참고)</mark>
- konlpy 도큐먼트: https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class
- OKT 도큐먼트: https://github.com/open-korean-text/open-korean-text

In [8]:
##########################################################################################
# Empty Module #2
# 입력: 자연어 상태의 리뷰 데이터
# 출력: 토큰화와 과정을 거쳐 단어들의 리스트로 변환된 데이터
# 입력 예시: "커피는 역시 학생회관 커피"
# 출력 예시: ["커피", "는", "역시", "학생", "회관", "커피"]
##########################################################################################
from konlpy.tag import Okt
okt = Okt()

def tokenize_words(sentence):
    sentence_tokenized = okt.morphs(sentence, norm=True, stem=True)#morphs: 입력받은 문자를 형태소 분석, norm: 정규화, stem: 어간 추출
    return sentence_tokenized

In [9]:
# 약 10-15분 정도 소요됩니다. 
x_train_tokenized = [tokenize_words(x) for x in tqdm(x_train_preprocessed, desc="tokenizing data")]

tokenizing data: 100%|██████████| 149993/149993 [12:29<00:00, 200.11it/s]


## \[Empty Module #3\] 불용어 제거
- 조사를 비롯한 불용어들은 많은 횟수 등장하지만, 리뷰의 긍정과 부정 여부를 판단하는데는 도움이 되지 않습니다.
- 데이터에서 아래 리스트로 정의된 불용어들을 제거해줍니다.

In [10]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']  #별다른 의미가 없는 불용어들

def exclude_stopwords(text):
    # 위 리스트에 포함된 불용어들을 제거하는 코드 작성
    ex_stop_text = [word for word in text if word not in stopwords] #리스트 컴프리헨션을 이용해 stopwords에 포함되지 않는 단어들만 리스트에 저장
    return ex_stop_text

x_train_stopwords_excluded = [exclude_stopwords(x) for x in x_train_tokenized]

## \[Empty Module #4\] 단어 임베딩
### 단어 임베딩 코드 구현
- 토큰화를 거쳐 분리된 단어들을 하나의 정수 값으로 매핑해주는 희소 표현법을 직접 구현해봅시다.
- 입력된 단어가 새로운 단어라면 새로운 정수 값을 할당하고, 이전에 등장한 단어라면 이전에 할당한 정수를 할당하는 함수를 작성합니다.
  - 단, <mark>테스트 데이터에 대해서는 새로운 단어가 등장하면 값을 할당하지 않습니다.</mark>

In [11]:
##########################################################################################
# Empty Module #4
# 입력: 단어 토큰화된 데이터
# 출력: 임베딩 과정을 거쳐, 각 단어가 하나의 실수 값으로 표현된 데이터
# 입력 예시: ["커피", "역시", "학생", "회관", "커피"]
# 출력 예시: [0, 1, 2, 3, 0]
##########################################################################################

embedding_dict = dict()  # 단어 임베딩을 위한 딕셔너리
embedding_value = 0

def embed_tokens(sentence_tokenized, mode):
    assert mode.upper() in ["TRAIN", "TEST"]
    global embedding_value
    
    sentence_embedded = list()#반환값을 저장할 리스트 생성
    for word in sentence_tokenized:
        # 코드 작성
        if mode == "TRAIN": #train 모드일때
            if word not in embedding_dict.keys(): #embedding_dict에 없는 새로운 단어라면
                embedding_dict[word] = embedding_value #embedding_dict에 word를 키, embedding_value를 value로 추가
                embedding_value += 1 #embedding_value에 값을 추가
            sentence_embedded.append(embedding_dict[word]) #sentence_embedded에 word의 embedding_value를 순서대로 저장
        else: #test 모드일때
            if word in embedding_dict.keys(): #embedding_dict에 단어가 존재할 때만
                sentence_embedded.append(embedding_dict[word]) #sentence_embedded에 word의 embedding_value를 순서대로 저장
            else: pass
    
    return sentence_embedded

In [12]:
# 실행 시간이 제법 소요됩니다. 비정상이 아니니 걱정하지 않으셔도 됩니다.
x_train_embedded = [embed_tokens(x, mode="TRAIN") for x in tqdm(x_train_stopwords_excluded, desc="embedding data")]
print(x_train_embedded[:5])
print("총 %d개의 단어가 임베딩되었습니다."%(embedding_value))

embedding data: 100%|██████████| 149993/149993 [00:01<00:00, 139563.87it/s]

[[0, 1, 2, 3, 4, 5], [6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19, 20], [21, 4, 14]]
총 44229개의 단어가 임베딩되었습니다.


## \[Empty Module #5\] 문장 벡터화
### Bag of Words 방법을 사용한 문장 벡터화
- 캐글 프로젝트 설명 페이지의 Bag of Words 방법 설명을 참고하여 Bag of Words 방법을 직접 구현해봅시다.

In [13]:
##########################################################################################
# Empty Module #5
# 입력: 임베딩 과정을 거친 데이터
# 출력: BoW 형태로 변환되어, M차원의 고정된 크기를 가진 벡터로 변환된 데이터
# 힌트: np.zeros((2, 3))는 [2, 3] 크기의 0으로 가득 찬 행렬을 생성합니다.
##########################################################################################
M = embedding_value# 전체 단어의 수

def to_BoW_representation(x):
    shape = (len(x),M)# BoW는 어떤 shape를 가져야 할까요? BoW는 행으로 전체 doc의 개수를 열로 전체 단어의 수를 가져야 합니다.
    x_BoW = np.zeros(shape)
    for i in tqdm(range(len(x)), desc="making BoW representation"):
        # 여기에 BoW 구현
        for word in x[i]:#하나의 doc list 안에 있는 word index를 가지고
            x_BoW[i, word] += 1 #해당하는 doc의 word columns 값에 맞는 자리에 1을 더해서 개수를 알려주겠다
    return x_BoW
x_train_BoW = to_BoW_representation(x_train_embedded)

making BoW representation: 100%|██████████| 149993/149993 [00:02<00:00, 62726.32it/s]


## \[Empty Module #6\] 차원 축소
- 우리가 만든 BoW는 수많은 리뷰에 등장하는 모든 단어들을 사용하여 만들어졌기 때문에, 엄청난 양의 단어들을 가지고 있습니다.
- 그러나, 실제로 영화 리뷰에 쓰이는 단어들은 이보다 적기 때문에, 많은 단어들이 전체 데이터에서 실제로는 한번도 등장하지 않거나, 매우 조금 등장하면서 공간을 차지하고 있을 것 입니다.
- 데이터의 크기를 줄여 머신러닝 모델이 중요한 정보에 집중할 수 있도록 해봅시다.
- <mark>학습 데이터에서 50번 미만으로 등장한 단어들을 제외</mark>해줍니다.

In [14]:
##########################################################################################
# Empty Module #6
# 입력: BoW 형태로 변환된 (N, M) 크기의 데이터
# 출력: 등장 빈도가 적은 단어들을 제외한 (N, m) 크기의 더 작은 데이터
##########################################################################################

In [15]:
# 힌트
# 1. 먼저 전체 데이터에서 각 단어가 등장한 횟수를 세어보세요.
# 2. 그 다음, 등장 횟수가 50회 미만인 단어들을 찾습니다.
# 3. 해당 단어들을 데이터에서 제거하는 코드를 작성합니다.
# 4. 설계를 잘 하고 구현을 시작해야 어렵지 않습니다.

In [16]:
# 여기에 코드 작성
save_columns = list() #뽑아낼 열 index를 저장할 리스트
for word_num in tqdm(range(embedding_value)): #각 열의 값을 모두 더하면
    count = np.sum(x_train_BoW[:,word_num]) #해당 열에 해당하는 단어가 총 몇 번 나왔는지 알 수 있음
    if count >= 50: #50개 이상일 경우
        save_columns.append(word_num)#앞서 언급한 리스트에 해당 열 index를 저장

x_train_BoW_reduced = x_train_BoW[:,save_columns]#해당 열 index에 맞는 열만 있는 차원이 축소된 새로운 데이터 생성
print("원본 BoW 크기:", x_train_BoW.shape)
print("차원 축소 후 크기:", x_train_BoW_reduced.shape)

100%|██████████| 44229/44229 [01:20<00:00, 546.87it/s]


원본 BoW 크기: (149993, 44229)
차원 축소 후 크기: (149993, 3112)


## \[Empty Module #7\]  분류 수행 및 제출: Bag of Words
- 이제 모든 문장이 고정된 크기 $m$ 차원의 벡터로 변환되었습니다.
- 원하는 모델을 사용하여, 각 문장의 영화에 대한 긍정적인 리뷰인지, 부정적인 리뷰인지 분류해봅시다.(Baseline 모델은 로지스틱 회귀입니다)
- 그 다음, 결과를 기록하여 Kaggle에 제출해봅시다!

In [17]:
##########################################################################################
# Empty Module #7
# 지금까지 전처리한 데이터로 분류를 수행하여, kaggle에 제출해봅시다.
# Baseline은 로지스틱 회귀입니다.
##########################################################################################
# 분류기 정의 및 학습 수행 코드 작성

In [18]:
# TEST 데이터를 전처리
x_test = test_data["review"]
x_test_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_test.iteritems(), total=len(x_test), desc="pre-processing data")]
x_test_tokenized = [tokenize_words(x) for x in tqdm(x_test_preprocessed, desc="tokenizing data")]
x_test_stopwords_excluded = [exclude_stopwords(x) for x in x_test_tokenized]
x_test_embedded = [embed_tokens(x, mode="TEST") for x in tqdm(x_test_stopwords_excluded, desc="embedding data")]
x_test_BoW = to_BoW_representation(x_test_embedded)
x_test_BoW_reduced = x_test_BoW[:,save_columns]

/tmp/ipykernel_20/2702516150.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  x_test_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_test.iteritems(), total=len(x_test), desc="pre-processing data")]
making BoW representation: 100%|██████████| 49999/49999 [00:00<00:00, 60234.19it/s]


In [19]:
# TEST 데이터에 대한 예측 수행 코드 작성
from sklearn.linear_model import LogisticRegression #로지스틱 회귀를 통해 예측할 것이다.
model = LogisticRegression(max_iter = 1000) #수렴할 수 있도록 충분한 iteration값을 줌
model.fit(x_train_BoW_reduced,y_train)#학습데이터로 학습
predict = model.predict(x_test_BoW_reduced)#테스트 데이터로 예측

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [20]:
submit = pd.read_csv("/kaggle/input/2023-ml-project1/sample_submission.csv", index_col=0)
# TEST 데이터에 대한 예측 값을 csv로 저장하는 코드 작성
submit['rating'] = predict #예측한 값 저장
submit.to_csv("bow_submit.csv",header=True,index=True)#예측한 값 csv로 저장

## \[Empty Module #8\] TF-IDF 적용
- BoW에서는 고려하지 않는 각 단어들의 중요도를 고려하기 위해, TF-IDF를 적용해봅시다.
- 캐글 프로젝트 설명 페이지의 설명을 참고하여 빈칸을 채워, TF-IDF를 구현해봅시다.

In [21]:
##########################################################################################
# Empty Module #8
# 빈칸을 적절히 채워넣어 TF-IDF를 위한 Inverse Document Frequency를 계산해봅시다.
##########################################################################################
N = len(x_train_BoW)  # 총 데이터 샘플의 수

def calculate_document_frequency(x):
    # 어떤 단어가 등장하는 데이터 샘플(문서)의 수(DF)를 계산하는 코드 작성
    x=np.array(x)#들어오는 데이터를 넘파이 배열로 변경
    document_frequency=[]#df를 저장할 리스트 생성
    for inx in tqdm(range(x.shape[1])):#데이터의 열값, 즉 단어에 따라
        DF = len([doc for doc in x[:,inx] if doc != 0])#문서에 해당하는 값이 0이 아니라면 리스트에 저장하고 0이 아닌 값들의 개수를 세어줌.
        document_frequency.append(DF)#열 순서로 저장
    return document_frequency#반환
    
def calculate_inverse_document_frequency(document_frequency):
    # DF에 반비례하는 IDF를 계산하는 코드 작성
    return [np.log(N/(df+1)) for df in document_frequency]#각 열에 따라 IDF를 계산하는 수식을 대입한 값을 반환

document_frequency = calculate_document_frequency(x_train_BoW_reduced) # 여기에 코드 작성
inverse_document_frequency = calculate_inverse_document_frequency(document_frequency) # 여기에 코드 작성

100%|██████████| 3112/3112 [00:36<00:00, 86.41it/s]


In [22]:
x_train_tfidf = np.array([x_train_BoW_reduced[:,indx]*inverse_document_frequency[indx] for indx in range(len(inverse_document_frequency))]).T # 해당 단어에 맞게 데이터에 IDF값을 곱해줌.

## \[Empty Module #9\]  분류 수행 및 제출: TF-IDF
- TF-IDF를 적용한 결과를 기록하여 Kaggle에 제출해봅시다!

In [23]:
##########################################################################################
# Empty Module #9
# TEST 데이터에 TF-IDF를 적용하여 모델을 학습, 예측을 수행하고 kaggle에 제출해봅시다.
# 이때, BoW와 같은 모델을 사용하여 성능을 비교해봅시다.
##########################################################################################
# 분류기 정의 및 학습 수행 코드 작성
model = LogisticRegression(max_iter = 1000) #수렴할 수 있도록 충분한 iteration값을 줌
model.fit(x_train_tfidf,y_train) #학습데이터로 학습

LogisticRegression(max_iter=1000)

In [24]:
# TEST 데이터를 전처리하는 코드 작성
test_document_frequency = calculate_document_frequency(x_test_BoW_reduced) # 여기에 코드 작성
test_inverse_document_frequency = calculate_inverse_document_frequency(document_frequency) # 여기에 코드 작성
x_test_tfidf = np.array([x_test_BoW_reduced[:,indx]*test_inverse_document_frequency[indx] for indx in range(len(test_inverse_document_frequency))]).T# 데이터에 위에서 구한 IDF를 곱하는 코드 작성

100%|██████████| 3112/3112 [00:12<00:00, 257.60it/s]


In [25]:
# 예측을 수행하는 코드 작성
predict = model.predict(x_test_tfidf)#테스트 데이터로 예측

In [26]:
submit = pd.read_csv("/kaggle/input/2023-ml-project1/sample_submission.csv", index_col=0)
# TEST 데이터에 대한 예측 값을 csv로 저장하는 코드 작성
submit['rating'] = predict #예측한 값 저장
submit.to_csv("tfidf_submit.csv",header=True,index=True)#예측한 값 csv로 저장

## 결과 비교 TIP
- BoW와 TF-IDF에 같은 모델을 적용하여 성능의 차이를 비교해봅시다.
- 각각의 방법론에 여러가지 모델을 적용하며, 사용한 방법에 따라 더 적절한 모델이 있는지 고려해봅시다.
- 실험 결과들을 토대로, 왜 이런 결과가 도출되었는지 고민해봅시다.

In [27]:
# from sklearn.naive_bayes import BernoulliNB
# model = BernoulliNB()
# model.fit(x_train_tfidf,y_train)
# predict = model.predict(x_test_tfidf)

# submit = pd.read_csv("/kaggle/input/2023-ml-project1/sample_submission.csv", index_col=0)
# # TEST 데이터에 대한 예측 값을 csv로 저장하는 코드 작성
# submit['rating'] = predict
# submit.to_csv("tfidf_naive_submit.csv",header=True,index=True)

In [28]:
# from sklearn.naive_bayes import BernoulliNB
# model = BernoulliNB()
# model.fit(x_train_BoW_reduced,y_train)
# predict = model.predict(x_test_BoW_reduced)

# submit = pd.read_csv("/kaggle/input/2023-ml-project1/sample_submission.csv", index_col=0)
# # TEST 데이터에 대한 예측 값을 csv로 저장하는 코드 작성
# submit['rating'] = predict
# submit.to_csv("naive_submit.csv",header=True,index=True)

In [29]:
# from sklearn.neighbors import KNeighborsClassifier
# model = KNeighborsClassifier()
# model.fit(x_train_BoW_reduced,y_train)
# predict = model.predict(x_test_BoW_reduced)

# submit = pd.read_csv("/kaggle/input/2023-ml-project1/sample_submission.csv", index_col=0)
# # TEST 데이터에 대한 예측 값을 csv로 저장하는 코드 작성
# submit['rating'] = predict
# submit.to_csv("bow_knc_submit.csv",header=True,index=True)